# TC viewer

IBTrACs database

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
import xarray as xr
import requests
from io import BytesIO
import numpy as np

sys.path.append("../../../functions")
from tcs import Extract_Circle
from data_downloaders import download_ibtracs
# from plot_tcs_alba import Map

sys.path.append("../../../../indicators_setup")
from ind_setup.plotting_tcs import Map


In [3]:
lon_lat = [134.5, 5.5] #Palau location lon, lat
basin = 'WP'#'SP'
r1 = 5 # Radius of the circular area in degrees

## Get Data

In [4]:
update_data = False
path_data = "../../../data"

In [5]:
if update_data:
    url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r01/access/netcdf/IBTrACS.ALL.v04r01.nc'
    tcs = download_ibtracs(url, basin = basin)
    tcs.to_netcdf(f"{path_data}/tcs_{basin}.nc")
else:
    tcs = xr.load_dataset(f"{path_data}/tcs_{basin}.nc")

In [6]:
d_vns = {
    'longitude': 'lon',
    'latitude': 'lat',
    'time': 'time',
    'pressure': 'wmo_pres',
}
tcs_sel, tcs_sel_params = Extract_Circle(tcs, lon_lat[0], lon_lat[1], r1, d_vns)
tcs_sel['name'].values = [i.decode('utf-8') for i in tcs_sel['name'].values]
tcs_sel['year'] = tcs_sel.time.dt.year

In [7]:
tcs_1979 = tcs_sel.where(tcs_sel_params.dmin_date.dt.year >=1979, drop = True)

## All TCs

In [8]:

mapHZ = Map()
fig = mapHZ.tcs_plotly(tcs_sel, lon_lat[0], lon_lat[1])
fig.update_layout(title=f"All TCs within {r1} degrees of {lon_lat[0]}E, {lon_lat[1]}N")
fig.update_layout(
    width=800,  # Ancho de la figura en píxeles
    height=600  # Altura de la figura en píxeles
)
fig.show()


## From 1979

In [9]:

mapHZ = Map()
fig = mapHZ.tcs_plotly(tcs_1979, lon_lat[0], lon_lat[1])
fig.update_layout(title=f"All TCs within {r1} degrees of {lon_lat[0]}E, {lon_lat[1]}N")
fig.update_layout(
    width=800,  # Ancho de la figura en píxeles
    height=600  # Altura de la figura en píxeles
)
fig.show()

## Severe TCs

Categories 3, 4 and 5

In [10]:
tcs_cat = tcs_sel.where(tcs_sel_params.category >= 3, drop = True)

In [11]:

mapHZ = Map()
fig = mapHZ.tcs_plotly(tcs_cat, lon_lat[0], lon_lat[1])
fig.update_layout(title=f"All TCs within {r1} degrees of {lon_lat[0]}E, {lon_lat[1]}N")
fig.update_layout(
    width=800,  # Ancho de la figura en píxeles
    height=600  # Altura de la figura en píxeles
)
fig.show()